In [1]:
import duckdb
import xml.etree.ElementTree as ET
# Import classifier
from ipynb.fs.full.group_classifier import predict_batch
print("Import successful!!")

/Users/marisbuttmann/PycharmProjects/stance-detection-german-llm/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Import successful
Import successful!!


In [2]:
 # Connect to sql database
con = duckdb.connect(database='../data/database/german-parliament', read_only=False)

In [3]:
def preprocess_speech(speech:tuple) -> tuple[int, list[str]]:
    # We only need the speech it to later identify the origin speech
    speech_id = test_speech[0]
    text = test_speech[5]
    # Parse xml from string will throw ParseError if not parseable
    root = ET.fromstring(text)
    # We only need paragraphs -> this filters unecessary information, such as comments or applause
    paragraphs = root.findall('p')
    paragraphs_text = []
    # Get text of paragraphs
    for p in paragraphs:
        # Use this to filter out speaker information
        if p.attrib.get("klasse") == "redner":
            continue
        else:
            p_text = p.text.strip()
            item = {"paragraphs":p_text}
            paragraphs_text.append(item) #get text and remove potential irrelevant whitespaces

    return (speech_id, paragraphs_text)
    

In [6]:
test_speech = con.sql("select * from speech order by random() limit 1 ").fetchone()

In [7]:
speech_id, paragraphs = preprocess_speech(test_speech)

print(speech_id)
print(paragraphs)

6600
[{'paragraphs': ['Meine Damen und Herren, die Beantwortung der Interpellation ist erfolgt. Ich frage, ob eine Besprechung der Interpellation gewünscht wird. — Das ist nicht der Fall. Damit ist Punkt 1 der Tagesordnung erledigt. Ich rufe auf Punkt 2 der Tagesordnung:']}, {'paragraphs': ['Erste Beratung des Entwurfs eines Gesetzes zur Regelung der Besteuerung des Kleinpflanzertabaks im Erntejahr 1950 (Nr. 1508 der Drucksachen).']}, {'paragraphs': ['Dazu hat zunächst das Wort Herr Staatssekretär Hartmann.']}]


In [2]:
paragraphs = [{'paragraphs': 'Meine Damen und Herren, die Beantwortung der Interpellation ist erfolgt. Ich frage, ob eine Besprechung der Interpellation gewünscht wird. — Das ist nicht der Fall. Damit ist Punkt 1 der Tagesordnung erledigt. Ich rufe auf Punkt 2 der Tagesordnung:'}, {'paragraphs': 'Erste Beratung des Entwurfs eines Gesetzes zur Regelung der Besteuerung des Kleinpflanzertabaks im Erntejahr 1950 (Nr. 1508 der Drucksachen).'}, {'paragraphs': 'Dazu hat zunächst das Wort Herr Staatssekretär Hartmann.'}]

classified_paragraphs = predict_batch(paragraphs)

Map: 100%|██████████| 3/3 [00:00<00:00, 568.41 examples/s]

Batch size: 3


In [7]:
for index, p in enumerate(classified_paragraphs):
    print(index, ": ",p, "\n\n")

0 :  [('[CLS]', 'O'), ('Meine', 'O'), ('Damen', 'O'), ('und', 'O'), ('Herren', 'O'), (',', 'O'), ('die', 'O'), ('Beantwortung', 'O'), ('der', 'O'), ('Inter', 'O'), ('##pel', 'O'), ('##lat', 'O'), ('##ion', 'O'), ('ist', 'O'), ('erfolgt', 'O'), ('.', 'O'), ('Ich', 'O'), ('frag', 'O'), ('##e', 'O'), (',', 'O'), ('ob', 'O'), ('eine', 'O'), ('Besprech', 'O'), ('##ung', 'O'), ('der', 'O'), ('Inter', 'O'), ('##pel', 'O'), ('##lat', 'O'), ('##ion', 'O'), ('gewünscht', 'O'), ('wird', 'O'), ('.', 'O'), ('[UNK]', 'O'), ('Das', 'O'), ('ist', 'O'), ('nicht', 'O'), ('der', 'O'), ('Fall', 'O'), ('.', 'O'), ('Damit', 'O'), ('ist', 'O'), ('Punkt', 'O'), ('1', 'O'), ('der', 'O'), ('Tages', 'O'), ('##ordnung', 'O'), ('erledigt', 'O'), ('.', 'O'), ('Ich', 'O'), ('ru', 'O'), ('##fe', 'O'), ('auf', 'O'), ('Punkt', 'O'), ('2', 'O'), ('der', 'O'), ('Tages', 'O'), ('##ordnung', 'O'), (':', 'O'), ('[SEP]', 'O')] 


1 :  [('[CLS]', 'O'), ('Erste', 'O'), ('Beratung', 'O'), ('des', 'O'), ('Entwurf', 'O'), ('##s',

In [12]:
p = classified_paragraphs[2]
groups = []
group_tmp = []
new_group = False
for token, label in p:
    if label == 'O':
        if group_tmp:
            groups.append(group_tmp)
            group_tmp = []
        new_group = True
        continue
    if new_group:
        new_group = False
    group_tmp.append((token, label))

print(groups)

[[('Herr', 'B-EPPOL'), ('Staatssekretär', 'I-EPPOL'), ('Hartmann', 'I-EPPOL')], [('[PAD]', 'I-EPPOL')], [('[PAD]', 'I-EPPOL')]]


In [53]:
nlp = spacy.blank("de")
words = [token for token, label in groups[7]]
print(words)
doc = Doc(nlp.vocab, words=words)
text = doc.text
print(text)

['Frauen', ',', 'die', 'unter', 'allen', 'Umständen', 'entschlossen', 'sind', ',', 'die']
Frauen , die unter allen Umständen entschlossen sind , die 
